In [1]:
import numpy as np
import pandas as pd
import pickle as pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [2]:
#  this is where the functions live !

# breaks up traffic into normal and attack
def separate_traffic_cats(df):
    normals = df.drop(df[df['label'] == 1].index).reset_index(drop=True)
    attacks = df.drop(df[df['label'] == 0].index).reset_index(drop=True)
    return normals, attacks


# one hot encodes and makes sure the cols are the correct categories
def preprocess(df, features):
    df['is_ftp_login'] = np.where(df['is_ftp_login'] > 1, 1, df['is_ftp_login'])
    df['service'].replace('-', np.nan, inplace=True)
    features['Type '] = features['Type '].str.lower()
    features['Name'] = features['Name'].str.lower()
    integer_names = features['Name'][features['Type '] == 'integer']
    binary_names = features['Name'][features['Type '] == 'binary']
    float_names = features['Name'][features['Type '] == 'float']
    cols = df.columns
    integer_names = cols.intersection(integer_names)
    binary_names = cols.intersection(binary_names)
    float_names = cols.intersection(float_names)

    for c in integer_names:
        pd.to_numeric(df[c])
    for c in binary_names:
        pd.to_numeric(df[c])
    for c in float_names:
        pd.to_numeric(df[c])

    num_col = df.select_dtypes(include='number').columns

    # selecting categorical data attributes
    cat_col = df.columns.difference(num_col)

    data_cat = df[cat_col].copy()
    data_cat = pd.get_dummies(data_cat, columns=cat_col)
    df = pd.concat([df, data_cat], axis=1)
    df.drop(columns=cat_col, inplace=True)
    df.replace([False, True], [0, 1], inplace = True) # look this over one more time
    return df


# removes the ground truth from the df
def remove_ground_truth(df, no_copy=True):  # set no_copy to False to copy the make a copy, True for in-place change
    ground_truth = df[['attack_cat', 'label']]
    df.drop(['attack_cat'], axis=1, inplace=no_copy)
    df.drop(['label'], axis=1, inplace=no_copy)
    return df, ground_truth


# a function that maps the vector from hdbscan onto the df to create the benign sub categories
def hdbscan_df_transformer(pickle_file, df):
    model = pickle.load(open(pickle_file, 'rb'))
    # extracting the sample and cluster sizes from the pickle file name
    file_list = list(filter(lambda x: x.isdigit(), pickle_file.split('_')))
    params = [int(s) for s in file_list]
    scan = pd.DataFrame(model, columns = [f'attack_cat_bc_{params[0]}_{params[1]}'])
    # print(scan.shape)
    # print(scan.head(500))
    # create the clustered columns with the parameters taken from the pickle file name
    df_concat = pd.concat([df, scan], axis = 1).reset_index(drop=True)
    # print('df_concat shape before concat: ' + str(df_concat.shape))
    df_concat[f'attack_cat_bc_{params[0]}_{params[1]}'] = 'Normal_' + df_concat[f'attack_cat_bc_{params[0]}_{params[1]}'].astype(str)
    df_concat[f'attack_cat_bc_{params[0]}_{params[1]}'].astype('category')
    # print('df_concat shape after concat: ' + str(df_concat.shape))
    return df_concat


# normalizes the passed in df
def normalization(df, col):
    minmax_scale = MinMaxScaler(feature_range=(0, 1))
    for i in col:
        if i == 'id':  # don't transform the ID column
            continue
        arr = df[i]
        arr = np.array(arr)
        df[i] = minmax_scale.fit_transform(arr.reshape(len(arr),1))
    return df


# splits the df into train and test and creates a column with the appropriate labels
def train_test_splitter(df):
    # make x the entire df
    x = df
    # the y can be anything since it doesn't get used to make the 'train_test' column
    y = df['label']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42, stratify=y)
    # once I have x_train and x_test, create a column in each called train_test and populate that column with either 0s or 1s
    x_train['train_test'] = 0
    x_test['train_test'] = 1
    # put train and test together
    df_concat = pd.concat([x_train, x_test], axis = 0).reset_index(drop=True)
    return df_concat

In [3]:
# pull in train + test + features
# train and test are incorrectly labelled but for consitency I will stick with it
df_test = pd.read_csv('../../wire/DataSets/UNSW-NB15/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv', index_col = 0)
df_train = pd.read_csv('../../wire/DataSets/UNSW-NB15/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv', index_col = 0)
features = pd.read_csv('../../wire/DataSets/UNSW-NB15/UNSW-NB15 - CSV Files/NUSW-NB15_features.csv', encoding='cp1252')

In [4]:
# concat train and test together
df_concat = pd.concat([df_train, df_test], axis = 0).reset_index(drop=True)

NameError: name 'df_train' is not defined

In [ ]:
# take out attack_cat and label so it does not get one hot encoded
df_train_noGT, train_GT = remove_ground_truth(df_train)
df_concat_noGT, concat_GT = remove_ground_truth(df_concat)

In [ ]:
# now we have 4 dfs
# first process train and concat dfs in preprocess() and pass in features
df_train_processed = preprocess(df_train_noGT, features)
df_train_nn = df_train_processed
df_concat_preprocessed = preprocess(df_concat_noGT, features)

In [ ]:
# next, normalize the preprocessed data
# leaving nn out makes it not normalized ('nn')
df_train_normalized = normalization(df_train_processed, df_train_processed.columns)
df_concat_normalized = normalization(df_concat_preprocessed, df_concat_preprocessed.columns)

In [ ]:
# now add the GT back into the dfs so that we can separate the normals from the attacks
# 82k normalized:
df_train_normalized_GT = pd.concat([df_train_normalized, train_GT], axis = 1)

# 82k not normalized
df_train_nn_GT = pd.concat([df_train_nn, train_GT], axis = 1)

# 250k normalized:
df_concat_normalized_GT = pd.concat([df_concat_normalized, concat_GT], axis = 1)

In [ ]:
# isolate the normal traffic from the attack traffic
# 82k normalized:
df_train_normal, df_train_attack = separate_traffic_cats(df_train_normalized_GT)

# 82k not normalized:
df_train_normal_nn, df_train_attack_nn = separate_traffic_cats(df_train_nn_GT)

# 250k normalized
df_concat_normal, df_concat_attack = separate_traffic_cats(df_concat_normalized_GT)

In [ ]:
# # map HDBSCAN into a new column attack_cat_bc using hdbscan_df_transformer
# # 82k normalized:
# df_train_normal = hdbscan_df_transformer('HDBSCANs_Normalized/HDBSCAN_normalized_normals_100_175', df_train_normal)
# df_train_normal.reset_index(inplace=True, drop=True)
# # in the df_train_attack df, duplicate 'attack_cat' into the new column attack_cat_bc
# df_train_attack['attack_cat_bc_100_175'] = df_train_attack['attack_cat']
# df_train_attack.reset_index(inplace=True, drop=True)

# # 82k not normalized:
# df_train_normal_nn = hdbscan_df_transformer('HDBSCANs_Not_Normalized/HDBSCAN_37k_Not_Normalized_100_175', df_train_normal_nn)
# df_train_normal_nn.reset_index(inplace=True, drop=True)
# # in the df_train_attack df, duplicate 'attack_cat' into the new column attack_cat_bc
# df_train_attack_nn['attack_cat_bc_100_175'] = df_train_attack_nn['attack_cat']
# df_train_attack_nn.reset_index(inplace=True, drop=True)

# # 250k normalized 100/500:
# df_concat_normal = hdbscan_df_transformer('Optimization/HDBSCANs_200-1000_93k/HDBSCAN_93k_100_500', df_concat_normal)
# df_concat_normal.reset_index(inplace=True, drop=True)
# # in the df_train_attack df, duplicate 'attack_cat' into the new column attack_cat_bc
# df_concat_attack['attack_cat_bc_100_500'] = df_concat_attack['attack_cat']
# df_concat_attack.reset_index(inplace=True, drop=True)

# # 250k normalized 250/5000:
# df_concat_normal_250_5000 = hdbscan_df_transformer('Optimization/HDBSCANs testing/HDBSCAN_93k_250_5000', df_concat_normal)
# df_concat_normal_250_5000.reset_index(inplace=True, drop=True)
# # in the df_train_attack df, duplicate 'attack_cat' into the new column attack_cat_bc
# df_concat_attack['attack_cat_bc_250_5000'] = df_concat_attack['attack_cat']
# df_concat_attack.reset_index(inplace=True, drop=True)

# 250k normalized 500/10000:
print('df_concat_normal shape: ' + str(df_concat_normal.shape))
df_concat_normal_500_10000 = hdbscan_df_transformer('HDBSCAN_93k_500_10000', df_concat_normal)
df_concat_normal_500_10000.reset_index(inplace=True, drop=True)
# # in the df_train_attack df, duplicate 'attack_cat' into the new column attack_cat_bc
df_concat_attack_500_10000 = df_concat_attack.copy()
df_concat_attack_500_10000['attack_cat_bc_500_10000'] = df_concat_attack_500_10000['attack_cat']
df_concat_attack_500_10000.reset_index(inplace=True, drop=True)
print('df_concat_normal_500_10000 shape: ' + str(df_concat_normal_500_10000.shape))
df_concat_normal_500_10000.head(500)

df_concat_normal shape: (93000, 197)
(93000, 1)
     attack_cat_bc_500_10000
0                          1
1                         -1
2                         -1
3                         -1
4                         -1
..                       ...
495                       -1
496                       -1
497                       -1
498                       -1
499                       -1

[500 rows x 1 columns]
df_concat shape before concat: (93000, 198)
df_concat shape after concat: (93000, 198)
df_concat_normal_500_10000 shape: (93000, 198)


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,proto_3pc,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_netblt,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pim,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_ptp,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rsvp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_sccopmce,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_sep,proto_skip,proto_sm,proto_smp,proto_snp,proto_sprite-rpc,proto_sps,proto_srp,proto_st2,proto_stp,proto_sun-nd,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_tp++,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no,attack_cat,label,attack_cat_bc_500_10000
0,1.833334e-07,0.000094,0.000000,0.000033,0.000000,0.090909,0.996078,0.000000,0.030121,0.000000,0.000000,0.000000,1.303758e-07,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.151351,0.000000,0.0,0.0,0.016129,0.333333,0.000000,0.000000,0.0,0.015625,0.0,0.0,0.0,0.000000,0.016393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Normal,0,Normal_1
1,1.333334e-07,0.000094,0.000000,0.000121,0.000000,0.125000,0.996078,0.000000,0.147128,0.000000,0.000000,0.000000,9.481875e-08,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.579054,0.000000,0.0,0.0,0.016129,0.333333,0.000000,0.000000,0.0,0.015625,0.0,0.0,0.0,0.000000,0.016393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
# # concat the normal traffic and attack traffic back together
# # 82k normalized:
# train_concat = pd.concat([df_train_normal, df_train_attack], axis = 0).reset_index(drop=True)

# # 82k not normalized:
# train_concat_nn = pd.concat([df_train_normal_nn, df_train_attack_nn], axis = 0).reset_index(drop=True)

# # 250k normalized 100/500:
# df_concat = pd.concat([df_concat_normal, df_concat_attack], axis = 0).reset_index(drop=True)

# # 250k normalized 250/5000:
# df_concat_250_5000 = pd.concat([df_concat_normal_250_5000, df_concat_attack], axis = 0).reset_index(drop=True)

# 250k normalized 500/10000:
df_concat_500_10000 = pd.concat([df_concat_normal_500_10000, df_concat_attack_500_10000], axis = 0).reset_index(drop=True)

In [ ]:
df_concat_500_10000.head(500)

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,proto_3pc,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_netblt,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pim,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_ptp,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rsvp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_sccopmce,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_sep,proto_skip,proto_sm,proto_smp,proto_snp,proto_sprite-rpc,proto_sps,proto_srp,proto_st2,proto_stp,proto_sun-nd,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_tp++,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no,attack_cat,label,attack_cat_bc_500_10000
0,1.833334e-07,0.000094,0.000000,0.000033,0.000000,0.090909,0.996078,0.000000,0.030121,0.000000,0.000000,0.000000,1.303758e-07,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.151351,0.000000,0.0,0.0,0.016129,0.333333,0.000000,0.000000,0.0,0.015625,0.0,0.0,0.0,0.000000,0.016393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Normal,0,Normal_1
1,1.333334e-07,0.000094,0.000000,0.000121,0.000000,0.125000,0.996078,0.000000,0.147128,0.000000,0.000000,0.000000,9.481875e-08,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.579054,0.000000,0.0,0.0,0.016129,0.333333,0.000000,0.000000,0.0,0.015625,0.0,0.0,0.0,0.000000,0.016393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
# split the data using train_test_splitter
# 82k normalized:
train_concat_ready = train_test_splitter(train_concat)

# 82k not normalized:
train_concat_nn_ready = train_test_splitter(train_concat_nn)

# 250k normalized:
df_concat_split = train_test_splitter(df_concat)

# 250 normalized 250/5000:
df_concat_250_5000_split = train_test_splitter(df_concat_250_5000)

# 250 normalized 500/10000:
df_concat_500_10000_split = train_test_splitter(df_concat_500_10000)

In [ ]:
# dump them to csv
train_concat_ready.to_csv('csv files/82k_normalized_cats_train_test_split.csv')
train_concat_nn_ready.to_csv('csv files/82k_not_normalized_cats_train_test_split.csv')
df_concat_split.to_csv('csv files/250k_normalized_cats_train_test_split_100_500.csv')
df_concat_250_5000_split.to_csv('csv files/250k_normalized_cats_train_test_split_250_5000.csv')
df_concat_500_10000_split.to_csv('csv files/250k_normalized_cats_train_test_split_500_10000.csv')